In [1]:
import fastai
from fastai import *          # Quick access to most common functionality
from fastai.vision import *   # Quick access to computer vision functionality
from fastai.layers import Lambda
from fastai.callbacks import *
import pytorch_ssim as ssim
from superres import *
from torchvision.models import vgg16_bn

In [2]:
path = Path('/DATA/WAMRI/salk/uri/Image_restoration_data/')
train_lr = path/'train_LR'
train_hr = path/'train_HR'
test_lr = path/'test_LR'
test_hr = path/'test_HR'

In [3]:
def pull_id(fn):
    return fn.split('#')[-1].split('.')[0]

def match_hr_fn(x):
    return hr_names_by_id[pull_id(x.name)]

lr_names_full = list(train_lr.glob('*.tif'))
lr_names_full.sort()
hr_names_by_id = {pull_id(hrfn.name):hrfn for hrfn in train_hr.glob('*.tif')}
test_label = list(hr_names_by_id.values())[0]

In [4]:
src = (GrayImageItemList
       .from_folder(train_lr, '*.tif', label_class=GrayImageItemList)
       .split_by_valid_func(lambda x: x.stem[-3] == '0')
       .label_from_func(match_hr_fn))

In [5]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

base_loss = F.l1_loss
vgg_m = vgg16_bn(True).features.cuda().eval()
requires_grad(vgg_m, False)
blocks = [i-1 for i,o in enumerate(children(vgg_m)) if isinstance(o,nn.MaxPool2d)]
blocks, [vgg_m[i] for i in blocks]

class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        self.wgts = layer_wgts
        self.metric_names = (['pixel',] + 
                             [f'feat_{i}' for i in range(len(layer_ids))] + 
                             [f'gram_{i}' for i in range(len(layer_ids))])

    def make_features(self, hooks, x, clone=False):
        self.m_feat(x.repeat([1,3,1,1]))
        return [(o.clone() if clone else o) for o in hooks.stored]
    
    def forward(self, input, target):
        hooks = hook_outputs(self.loss_features, detach=False)
        out_feat = self.make_features(hooks, target, clone=True)
        in_feat = self.make_features(hooks, input)
        hooks.remove()
        
        self.feat_losses = [F.mse_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)

feat_loss = FeatureLoss(vgg_m, blocks[2:5], [5,15,2])

In [6]:
scale=4
n_resblocks=16
n_feats=128
res_scale= 1.
model = WDSR(scale, n_resblocks, n_feats, res_scale, n_colors_in=1)
model = nn.DataParallel(model) #.eval().cuda()

In [7]:
sz_lr = 72
scale,bs = 4,24
sz_hr = sz_lr*scale
loss = combo_edge_mse
#loss=feat_proj_loss
#loss=combo2
loss = feat_loss

In [ ]:
# learn = build_learner(model, bs, sz_lr, sz_hr, src, loss=loss, callback_fns=LossMetrics)
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
epochs, lr = 10, 2e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, save='wdsr_1.0', loss=loss, callback_fns=[LossMetrics])

In [ ]:
torch.save(learn.model.module, "wdsr_model_1")

In [ ]:
sz_lr = 128
scale,bs = 4,8
sz_hr = sz_lr*scale
epochs, lr = 30, 1e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, load='wdsr_1.0_best', save='wdsr_1.1', loss=loss, callback_fns=[LossMetrics])

load: wdsr_1.0_best


epoch,train_loss,valid_loss,mse_loss,ssim,psnr
1,0.996772,0.882731,0.009071,0.692020,20.427378


In [ ]:
learn = learn.load('wdsr_1.1')
torch.save(learn.model.module,'wdsr_model_1.1')

In [ ]:
sz_lr = 256
scale,bs = 4,4
sz_hr = sz_lr*scale
epochs, lr = 50, 1e-3
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, load='wdsr_1.1', save='wdsr_1.2', loss=loss)

In [ ]:
learn = learn.load('wdsr_1.2')
torch.save(learn.model.module,'wdsr_model_1.2')

In [ ]:
sz_lr = 256
scale,bs = 4,4
sz_hr = sz_lr*scale
epochs, lr = 50, 1e-4
learn = batch_learn(model, bs, sz_lr, sz_hr, lr, epochs, src, load='wdsr_1.2', save='wdsr_1.3', loss=loss)

In [ ]:
learn = learn.load('wdsr_1.3')
torch.save(learn.model.module,'wdsr_model_1.3')

In [ ]:
learn = learn.load('wdsr_1.3_best')
torch.save(learn.model.module,'wdsr_model_1.3_best')

In [ ]:
learn = learn.load('wdsr_1.3_best')

In [ ]:
sz_lr = 506
scale,bs = 4,1
sz_hr = sz_lr*scale
data = get_data(src, bs, sz_lr, sz_hr)
learn = Learner(data, model, loss_func=F.mse_loss)
preds, ys = learn.get_preds(DatasetType.Valid)
preds = torch.clamp(preds, 0.,1.)

In [ ]:
idx = 9
ys[idx].std(), preds[idx].std()

In [ ]:
preds[idx].min(),preds[idx].max()

In [ ]:
hr = Image(ys[idx])
hr

In [ ]:
sr = Image(preds[idx])
sr

In [ ]:
lr = learn.data.valid_ds[idx][0]
lr

In [ ]:
ssim.ssim(sr.data[None],hr.data[None]), psnr(sr.data[None],hr.data[None]), F.mse_loss(sr.data[None],hr.data[None])

In [ ]:
learn.save('wdsr')

In [ ]:
learn = learn.load('wdsr')

In [ ]:
trn_tfm, val_tfm = get_transforms()
test_bs = 2
test_sz = 512 
test_out_sz = 4*test_sz
test_data = get_data(src, test_bs, test_sz, test_out_sz, test_folder=path/'newimg')
test_learn = Learner(test_data, model, loss_func=F.mse_loss).load('wdsr')

In [ ]:
test_preds, ys = test_learn.get_preds(DatasetType.Test)
test_preds = torch.clamp(test_preds, 0.,1.)

In [ ]:
sr_imgs = []
for idx in range(test_preds.shape[0]):
    img = Image(test_preds[idx])
    sr_imgs.append(img)

In [ ]:
sr_imgs[1]

In [ ]:
sr_imgs[0]

In [ ]:
test_learn.save('wdsr_keep_edge3')

In [ ]:
test_learn.save('wdsr_keep_edge3')